In [163]:
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error


import numpy as np

from xgboost import XGBRegressor

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

import datetime
import pickle

In [164]:
# from sklearn.model_selection import TimeSeriesSplit

In [165]:
full_df = pd.read_csv("../data/processed/fhvhv_daily_counts_features.csv")

full_df['pickup_date'] = pd.to_datetime(full_df['pickup_date'])
# set PULocationID as categorical
full_df["PULocationID"] = full_df["PULocationID"].astype("category")

full_df.head()

,PULocationID,pickup_date,count,day_of_week,day_of_month,month,is_weekend,lag_1,lag_7,rolling_mean_3,rolling_mean_7,rolling_mean_14,rolling_mean_30,rolling_median_3,rolling_median_7,rolling_median_14,rolling_median_30,rolling_std_3,rolling_std_7,rolling_std_14,rolling_std_30,rolling_min_3,rolling_min_7,rolling_min_14,rolling_min_30,rolling_max_3,rolling_max_7,rolling_max_14,rolling_max_30,expanding_mean,expanding_median,expanding_std,expanding_min,expanding_max,diff_1,diff_7,day_of_week_sin,day_of_week_cos
0,1,2022-02-01,0,1,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.201299,0.979530
1,1,2022-02-02,0,2,2,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.394356,0.918958
2,1,2022-02-03,0,3,3,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.571268,0.820763
3,1,2022-02-04,0,4,4,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.724793,0.688967
4,1,2022-02-05,0,5,5,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.848644,0.528964


In [16]:
pd.set_option('display.max_columns', None)

In [17]:
HOLDOUT_LENGTH_W = 1

HOLDOUT_DATE_START = full_df.pickup_date.max() - pd.Timedelta(weeks=HOLDOUT_LENGTH_W)

HOLDOUT_DATE_START

Timestamp('2023-09-23 00:00:00')

In [18]:
holdout_df = full_df[full_df.pickup_date > HOLDOUT_DATE_START]

df = full_df[full_df.pickup_date <= HOLDOUT_DATE_START]

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 157800 entries, 0 to 159633
Data columns (total 38 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   PULocationID       157800 non-null  category      
 1   pickup_date        157800 non-null  datetime64[ns]
 2   count              157800 non-null  int64         
 3   day_of_week        157800 non-null  int64         
 4   day_of_month       157800 non-null  int64         
 5   month              157800 non-null  int64         
 6   is_weekend         157800 non-null  int64         
 7   lag_1              157800 non-null  float64       
 8   lag_7              157800 non-null  float64       
 9   rolling_mean_3     157800 non-null  float64       
 10  rolling_mean_7     157800 non-null  float64       
 11  rolling_mean_14    157800 non-null  float64       
 12  rolling_mean_30    157800 non-null  float64       
 13  rolling_median_3   157800 non-null  float64      

In [20]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [66]:
BACKTESTS = 3
BACKTESTS_LENGTH_W = 1

def split_train_test_by_date(df, BACKTESTS=BACKTESTS, BACKTESTS_LENGTH_W=BACKTESTS_LENGTH_W):
    """
    Splits a dataframe into train and test sets by date. Create BACKTESTS number of backtests, each BACKTESTS_LENGTH_W weeks long.
    We will do moving window backtests
    """

    TRAIN_START = df.pickup_date.min() - pd.Timedelta(days=1)
    TRAIN_LENGTH = df.pickup_date.max() - pd.Timedelta(weeks=BACKTESTS_LENGTH_W*BACKTESTS) - TRAIN_START

    end = df.pickup_date.max()
    earliest_train_start = df.pickup_date.min()

    train_val_pairs = []
    for i in range(BACKTESTS):
        backtest_end = end - pd.Timedelta(days=1)  # End date of backtest is one day before holdout starts
        backtest_start = backtest_end - pd.Timedelta(weeks=BACKTESTS_LENGTH_W) + pd.Timedelta(days=1)

        train_start = earliest_train_start + pd.Timedelta(weeks=BACKTESTS_LENGTH_W*(BACKTESTS - i-1))
        train_end = backtest_start - pd.Timedelta(days=1)

        # print(f"Backtest {i+1}:")
        # print(f"   Training Data: Start - {train_start.date()}, End - {train_end.date()}")
        # print(f"   Backtest Data: Start - {backtest_start.date()}, End - {backtest_end.date()}")

        train_df = df[(df.pickup_date >= train_start) & (df.pickup_date <= train_end)]
        test_df = df[(df.pickup_date >= backtest_start) & (df.pickup_date <= backtest_end)]

        train_val_pairs.append((train_df, test_df))

        end = backtest_start

    return train_val_pairs

In [73]:
default_parameters = {'n_estimators':300, 'learning_rate':0.1, 'max_depth':5, 
                      'min_child_weight':1, 'gamma':0, 'subsample':1, 'colsample_bytree':0.3, 'alpha':10}

In [143]:
def train_xgb(train_df, test_df = None, parameters = default_parameters, ):

    early_stopping_rounds = None

    X_train = train_df.drop(columns=["pickup_date", "count"])
    y_train = train_df["count"]

    if test_df is not None:
        X_test = test_df.drop(columns=["pickup_date", "count"])
        y_test = test_df["count"]

        xg_reg = XGBRegressor(**parameters,
                          objective = "reg:squarederror", eval_metric = "rmse", seed = 123, n_jobs = -1, enable_categorical = True, 
                          early_stopping_rounds=10)

        xg_reg.fit(X_train,y_train, eval_set=[(X_train, y_train), (X_test, y_test)], verbose=False)

    else:
        xg_reg = XGBRegressor(**parameters,
                    objective = "reg:squarederror", eval_metric = "rmse", seed = 123, n_jobs = -1, enable_categorical = True)

        xg_reg.fit(X_train,y_train)

    return xg_reg

### Creating optimisation with hyperopt

In [149]:
search_space = {
    'learning_rate': hp.loguniform('learning_rate', -7, 0),
    'max_depth': hp.quniform('max_depth', 5, 15, 1),
    'min_child_weight': hp.loguniform('min_child_weight', -2, 3),
    'subsample': hp.uniform('subsample', 0.6, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1),
    'n_estimators': hp.quniform('n_estimators', 200, 750, 50),
    'gamma': hp.loguniform('gamma', -2, 3),
    'alpha': hp.loguniform('alpha', -2, 3),
    'lambda': hp.loguniform('lambda', -2, 3),
}

In [150]:
def objective(space):
    params = {
        'learning_rate': space['learning_rate'],
        'max_depth': int(space['max_depth']),
        'min_child_weight': space['min_child_weight'],
        'subsample': space['subsample'],
        'colsample_bytree': space['colsample_bytree'],
        'gamma': space['gamma'],
        'alpha': space['alpha'],
        'lambda': space['lambda'],
        'n_estimators': int(space['n_estimators']),
    }
    LOSS = []
    for j, (train, test) in enumerate(split_train_test_by_date(TRAIN_DF)):
        model_xgb_opt_cv = train_xgb(train, test, parameters=params)

        pred = model_xgb_opt_cv.predict(test.drop(columns=["pickup_date", "count"]))
    
        loss_model = rmse(test["count"], pred)
    
    LOSS.append(loss_model)

    loss = np.mean(LOSS)
    
    return {'loss': loss, 'status': STATUS_OK}

In [151]:
trials = Trials()  
n_iter = 100

## Training loop
We optimise the hyperparameters of the model using the hyperopt library.
The optimisation will happen on the latest (most recent) backtest.

Then we will just train the model with the best hyperparameters on the other backtests and evaluate it separately.


In [147]:
from hyperopt.early_stop import no_progress_loss

In [152]:
train, test = split_train_test_by_date(df)[0]

print("DEFAULT PARAMETERS ARE: {}".format(default_parameters))
        # do internal CV on first backtest
TRAIN_DF = train.copy()

best=fmin(fn=objective, space=search_space, algo=tpe.suggest,
max_evals=20,
trials=trials,
early_stop_fn=no_progress_loss(10)
)


DEFAULT PARAMETERS ARE: {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 1, 'gamma': 0, 'subsample': 1, 'colsample_bytree': 0.3, 'alpha': 10}
 70%|███████   | 14/20 [18:44<08:02, 80.34s/trial, best loss: 199.1131041015372]


In [154]:
best

{'alpha': 7.770483145979562,
 'colsample_bytree': 0.6212693715774648,
 'gamma': 7.446473540559165,
 'lambda': 0.580758896382699,
 'learning_rate': 0.015156138623413178,
 'max_depth': 14.0,
 'min_child_weight': 4.733033609230644,
 'n_estimators': 700.0,
 'subsample': 0.7801290200553479}

In [155]:
# copy best parameters to default parameters
best_parameters_final = best.copy() 
best_parameters_final['max_depth'] = int(best_parameters_final['max_depth'])
best_parameters_final['n_estimators'] = int(best_parameters_final['n_estimators'])
best_parameters_final

{'alpha': 7.770483145979562,
 'colsample_bytree': 0.6212693715774648,
 'gamma': 7.446473540559165,
 'lambda': 0.580758896382699,
 'learning_rate': 0.015156138623413178,
 'max_depth': 14,
 'min_child_weight': 4.733033609230644,
 'n_estimators': 700,
 'subsample': 0.7801290200553479}

In [156]:
# best_params_final = {
#  'alpha': 0.5646387212271088,
#  'colsample_bytree': 0.6585122987835552,
#  'gamma': 0.8069274373476782,
#  'lambda': 13.622702434901178,
#  'learning_rate': 0.05005038815036528,
#  'max_depth': 39,
#  'min_child_weight': 0.3787276602617831,
#  'n_estimators': 579,
#  'subsample': 0.9560746826764644}

In [157]:
for i, (train, test) in enumerate(split_train_test_by_date(df)):
    

    if i==0:

        print("MODEL IS TUNED. BEST HYPERPARAMETERS ARE: {}".format(best_parameters_final))

        tuned_model = train_xgb(train, test, best_parameters_final)
        prediction = tuned_model.predict(test.drop(columns=["pickup_date", "count"]))

        print("RMSE for backtest {} is {}".format(i, rmse(test['count'], prediction)))
        

    else:
        # model_xgb = train_xgb(train, best_parameters_final)

        prediction = tuned_model.predict(test.drop(columns=["pickup_date", "count"]))
        
        print("RMSE for backtest {} is {}".format(i, rmse(test['count'], prediction)))

MODEL IS TUNED. BEST HYPERPARAMETERS ARE: {'alpha': 7.770483145979562, 'colsample_bytree': 0.6212693715774648, 'gamma': 7.446473540559165, 'lambda': 0.580758896382699, 'learning_rate': 0.015156138623413178, 'max_depth': 14, 'min_child_weight': 4.733033609230644, 'n_estimators': 700, 'subsample': 0.7801290200553479}
RMSE for backtest 0 is 251.7098825972449
RMSE for backtest 1 is 118.94386613148271
RMSE for backtest 2 is 145.39392073141966


In [160]:
# train on all data to evaluate on holdout
final_model = train_xgb(df, test_df = None, parameters = best_parameters_final)

#predict on hodlout
holdout_prediction = final_model.predict(holdout_df.drop(columns=["pickup_date", "count"]))
print("RMSE for holdout is {}".format(rmse(holdout_df['count'], holdout_prediction)))

RMSE for holdout is 383.7053058807188


In [167]:
# train on full_df
final_model = train_xgb(full_df, parameters = best_parameters_final)


In [169]:
# save model using timestamp

final_model.save_model("../models/xgb_{}.json".format(datetime.datetime.now().strftime("%Y%m%d_%H%M%S")))